**Create a transformer from scratch using the Pytorch library **

In [1]:
#build a text classification model using a Transformer architecture.
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [2]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Sample dataset (replace with real dataset)
data = [
    ("The cat sat on the mat", 0),
    ("Dogs are loyal animals", 1),
    ("Transformers are powerful models", 1),
    ("Pytorch makes deep learning easy", 0),
]
vocab = list(set(word for sentence, _ in data for word in sentence.split()))
word2idx = {word: idx for idx, word in enumerate(vocab)} #word2idx dictionary is created to map words to numerical indices.

In [4]:
# Convert dataset into token indices
class TextDataset(Dataset):
    def __init__(self, data, word2idx, max_len=None):  # Add max_len parameter
        self.data = [(torch.tensor([word2idx[word] for word in sentence.split()], dtype=torch.long), label)
                     for sentence, label in data]
        self.max_len = max_len if max_len else max(len(x[0]) for x in self.data)  # Calculate or use provided max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Pad sequences to max_len
        tensor, label = self.data[idx]
        padded_tensor = torch.zeros(self.max_len, dtype=torch.long)
        padded_tensor[:len(tensor)] = tensor
        return padded_tensor, label  # Return padded tensor

dataset = TextDataset(data, word2idx)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [5]:
# Transformer Model Definition
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, hidden_dim, num_classes):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, seq_len, embed_dim)
        x = x.permute(1, 0, 2)  # Required shape for Transformer (seq_len, batch_size, embed_dim)
        x = self.transformer_encoder(x)  # Apply transformer
        x = x.mean(dim=0)  # Global Average Pooling
        x = self.fc(x)  # Fully connected layer
        return x

In [6]:
# Hyperparameters
vocab_size = len(vocab)
embed_dim = 32
num_heads = 2
num_layers = 2
hidden_dim = 64
num_classes = 2

In [7]:
# Model, Loss, Optimizer
model = TransformerModel(vocab_size, embed_dim, num_heads, num_layers, hidden_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [8]:
# Training Loop
def train_model(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

train_model(model, dataloader, criterion, optimizer)

Epoch 1/10, Loss: 1.3434
Epoch 2/10, Loss: 1.0821
Epoch 3/10, Loss: 0.9728
Epoch 4/10, Loss: 0.7832
Epoch 5/10, Loss: 0.6502
Epoch 6/10, Loss: 0.5789
Epoch 7/10, Loss: 0.4295
Epoch 8/10, Loss: 0.3740
Epoch 9/10, Loss: 0.3070
Epoch 10/10, Loss: 0.2494


In [9]:
# Save Model
torch.save(model.state_dict(), "transformer_model.pth")
print("Model saved successfully!")

Model saved successfully!
